In [47]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
#importing and fixing os due to error on MKL dataleak.  Currently just annoying but this fixes it.  
import os
# Import required libraries and dependencies
import pandas as pd
# import hvplot.pandas
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from pathlib import Path
# from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import ConfusionMatrixDisplay

In [48]:
pres_gdp = pd.read_csv("president_gdp-94.csv")
# pres_bday = pd.read_csv("president_birthday-94.csv")
pres_terms = pd.read_csv("president_terms-94.csv",encoding="UTF-8") 
# pres_bday=pres_bday.drop("csvbase_row_id",axis=1)

In [49]:
## EXPAND YEAR OF TERMS PER LINE

# Create a new DataFrame that will contain the expanded rows
expanded_rows = []

# Iterate over each row and duplicate the data for each year between Start and Stop
for i, row in pres_terms.iterrows():
    for year in range(row['Start'], row['Stop'] + 1):
        row_data = row.to_dict()  # Convert row to a dictionary
        row_data['Year'] = year   # Add the expanded year to the dictionary
        expanded_rows.append(row_data)


# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)

# Display the DataFrame (this would save to a file or display the result in your environment)
#expanded_df.to_csv('expanded_president_list.csv', index=False)

#print(expanded_df)
expanded_df.head()

,Start,Stop,Name,VP,BDay,BMonth,BYear,Year
0,1789,1797,George Washington,John Adams,22,2,1732,1789
1,1789,1797,George Washington,John Adams,22,2,1732,1790
2,1789,1797,George Washington,John Adams,22,2,1732,1791
3,1789,1797,George Washington,John Adams,22,2,1732,1792
4,1789,1797,George Washington,John Adams,22,2,1732,1793


In [50]:
expanded_df = pd.merge(expanded_df,pres_gdp, on = "Year")

expanded_df.head()

,Start,Stop,Name,VP,BDay,BMonth,BYear,Year,Party,First Name,...,Nominal GDP (million of Dollars),Real GDP (millions of 2017 dollars),GDP Deflator (index 2017=100),Population,Nominal GDP per capita (current dollars),Real GDP per capita (year 2017 dollars),GDP Growth,GDP Percent Growth,Population Growth,Population Percent Growth
0,1789,1797,George Washington,John Adams,22,2,1732,1789,Independent,George,...,176,"4,676",3.78,"3,810,000",46.31,"1,229.61",1,0.030,1,0.031
1,1789,1797,George Washington,John Adams,22,2,1732,1790,Independent,George,...,193,"4,975",3.88,"3,929,000",49.1,"1,266.23",1,0.029,1,0.030
2,1789,1797,George Washington,John Adams,22,2,1732,1791,Independent,George,...,210,"5,274",3.98,"4,048,000",51.89,"1,302.85",1,0.042,1,0.030
3,1789,1797,George Washington,John Adams,22,2,1732,1792,Independent,George,...,230,"5,663",4.05,"4,171,000",55.03,"1,357.63",1,0.048,1,0.030
4,1789,1797,George Washington,John Adams,22,2,1732,1793,Independent,George,...,256,"6,113",4.19,"4,297,000",59.67,"1,422.52",1,0.096,1,0.030


In [51]:
## LABEL ENCODE PARTY HERE then make it a numerical, then use it in the Y
## THIS IS WHERE WE WOULD LABEL ENCODE HOROSCOPE AS WELL

# Set index to 'Year' and select columns for categorical_df
categorical_df = expanded_df[['Year','Name', 'VP','BDay', 'Party', 'BMonth', 'GDP Growth', 'Population Growth']].set_index('Year')

# Select numerical columns for numerical_df
numerical_df = expanded_df[['Year','GDP Percent Growth', 'Population Percent Growth']].set_index('Year')

categorical_df.head(3)
# numerical_df



,Name,VP,BDay,Party,BMonth,GDP Growth,Population Growth
Year,,,,,,,
1789,George Washington,John Adams,22,Independent,2,1,1
1790,George Washington,John Adams,22,Independent,2,1,1
1791,George Washington,John Adams,22,Independent,2,1,1


In [52]:
categorical_df = pd.get_dummies(categorical_df)
categorical_df.head(3)

,BDay,BMonth,GDP Growth,Population Growth,Name_Abraham Lincoln,Name_Andrew Jackson,Name_Andrew Johnson,Name_Barack Obama,Name_Benjamin Harrison,Name_Calvin Coolidge,...,VP_William A. Wheeler,VP_William R. King,VP_office vacant,Party_Democrat,Party_Democratic-Republican,Party_Federalist,Party_Independent,Party_National Union,Party_Republican,Party_Whig
Year,,,,,,,,,,,,,,,,,,,,,
1789,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1790,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1791,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [53]:
# MERGEM THEM THANGS
dummy_df = pd.merge(categorical_df,numerical_df, left_index=True, right_index=True)

dummy_df

,BDay,BMonth,GDP Growth,Population Growth,Name_Abraham Lincoln,Name_Andrew Jackson,Name_Andrew Johnson,Name_Barack Obama,Name_Benjamin Harrison,Name_Calvin Coolidge,...,VP_office vacant,Party_Democrat,Party_Democratic-Republican,Party_Federalist,Party_Independent,Party_National Union,Party_Republican,Party_Whig,GDP Percent Growth,Population Percent Growth
Year,,,,,,,,,,,,,,,,,,,,,
1789,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.030,0.031
1790,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.029,0.030
1791,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.042,0.030
1792,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.048,0.030
1793,22,2,1,1,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,0.096,0.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,14,6,1,1,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,0.016,0.004
2021,20,11,1,1,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,0.016,0.004
2021,20,11,1,1,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,0.016,0.004


In [54]:
# Define the target columns (y)
y = dummy_df[['BDay', 'BMonth']]

# Exclude the columns in y from dummy_df3
X = dummy_df.loc[:, ~dummy_df.columns.isin(y.columns)]

In [55]:
print(X.shape)  # Should be (n_samples, n_features)
print(y.shape)  # Should be (n_samples,) or (n_samples, 1)

(501, 104)
(501, 2)


In [56]:
# Split the preprocessed data into a training and testing dataset -- choose to stratify so that the classes are more balanced out bc it's a small dataset and imbalanced
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2, random_state=42)

In [57]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler on the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [58]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model_logit = LogisticRegression(random_state=1)

# Fit the model using training data
model_logit.fit(X_train, y_train)

ValueError: y should be a 1d array, got an array of shape (400, 2) instead.

In [ ]:
# Make a prediction using the testing data
y_pred_logit = model_logit.predict(X_test)

pd.DataFrame({
    "actual": y_test,
    "predictions": y_pred_logit
}).head(7)

In [ ]:
#accuracy score
accuracy_score(y_true= y_test, y_pred= y_pred_logit)

In [ ]:
cm_df = pd.DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred_logit))

cm_df

In [ ]:
# Print the classification report for the model
print(classification_report(y_true = y_test, y_pred = y_pred_logit))